In [ ]:
# | default_exp server

In [ ]:
# | export

import importlib
import sys
import asyncio
from typing import *
from contextlib import contextmanager
from pathlib import Path
import threading
import signal
from contextlib import ExitStack, contextmanager
from tempfile import TemporaryDirectory
import subprocess


import multiprocessing
from fastcore.meta import delegates
from fastcore.basics import patch
import typer
from anyio import create_task_group

from fastkafka.application import FastKafka
from fastkafka.testing import change_dir
from fastkafka._components.helpers import _import_from_string, generate_app_src
from fastkafka._components.logger import get_logger

In [ ]:
import os
from time import sleep
from pydantic import BaseModel
from fastkafka._components.logger import supress_timestamps


In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
def create_test_app():
    app = FastKafka(bootstrap_servers="tvrtko-fastkafka-kafka-1:9092")

    class MyMessage(BaseModel):
        msg: str

    @app.consumes()
    def on_my_topic(msg: MyMessage):
        pass
    
    return app

In [ ]:
# | export

@contextmanager
def generate_app_in_tmp() -> Generator[None, None, None]:
    with TemporaryDirectory() as d:
        src_path = Path(d) / "main.py"
        generate_app_src(src_path)
        with change_dir(d):
            import_str = f"{src_path.stem}:kafka_app"
            yield import_str

In [ ]:
# | export


class ServerProcess:
    def __init__(self, app: str):
        self.app = app
        self.should_exit = False

    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        self.application = _import_from_string(self.app)

        async with self.application:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )

        def handle_exit(sig: int) -> None:
            self.should_exit = True

        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)

    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

def terminate_process(p: Union[subprocess.Popen]) -> Tuple[bytes, bytes]:
    for i in range(3):
        p.terminate()
        try:
            p.wait(timeout=10)
            return p.communicate()

        except subprocess.TimeoutExpired:
            logger.warning("Process not terminated, retrying...")

    logger.warning("Killing the process...")
    p.kill()
    p.wait()
    outs, errs = p.communicate()
    logger.warning("Process killed!")

    return outs, errs


_app = typer.Typer()


@_app.command()
def run_fastkafka_server_process(
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    )
) -> None:
    ServerProcess(app).run()

In [ ]:
# | notest

print("WARNING: make sure you save the notebook before running this cell\n")

print("Exporting and installing the new version of the CLI command...")
subprocess.run(["nbdev_export"], check=True, capture_output=True)
subprocess.run(["pip", "install", "-e", "..[dev]"], check=True, capture_output=True)

print("ok")


Exporting and installing the new version of the CLI command...
ok


In [ ]:
#| export


async def terminate_asyncio_process(p: asyncio.subprocess.Process) -> Tuple[bytes, bytes]:
    for i in range(3):
        p.terminate()
        try:
            return await asyncio.wait_for(p.communicate(), 10)
        
        except subprocess.TimeoutExpired:
            logger.warning("Process not terminated, retrying...")

    logger.warning("Killing the process...")
    p.kill()
    outs, errs = await p.communicate()
    logger.warning("Process killed!")

    return outs, errs

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
                "run_fastkafka_server_process", app, stdout=asyncio.subprocess.PIPE
            )
    sleep(5)
    outputs, _ = await terminate_asyncio_process(proc)
    
    print(outputs.decode("utf-8"))
    assert proc.returncode == 0

[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpdpt9uggj/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpdpt9uggj/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_cons

In [ ]:
# | export


async def run_fastkafka_server(num_workers: int, app: str):
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )

    d = {"should_exit": False}

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        loop.add_signal_handler(sig, handle_exit, sig)

    procs = [
        await asyncio.create_subprocess_exec(
            "run_fastkafka_server_process", app, stdout=asyncio.subprocess.PIPE
        )
        for i in range(num_workers)
    ]
    
    async def log_output(output: asyncio.StreamReader, worker_id: int, d: Dict[str, bool] = d) -> None:
        while not d["should_exit"]:
            outs = await output.read(n=1024)
            if outs != b"":
                typer.echo(f"[{worker_id:03d}]: " + outs.decode("utf-8"), nl=False)
    
    async with create_task_group() as tg:
        log_tasks = [tg.start_soon(log_output, proc.stdout, i) for i, proc in enumerate(procs)]
        while not d["should_exit"]:
            await asyncio.sleep(0.2)

#     terminate and stuff
#     outputs = [await terminate_asyncio_process(proc) for proc in procs]
    [await proc.wait() for proc in procs]
    outputs = [await proc.communicate() for proc in procs]

    for i, (output, _) in enumerate(outputs):
        typer.echo(f"[{i:03d}]: " + output.decode("utf-8"))

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | notest

with generate_app_in_tmp() as app:
    print(f"{app=}")
    await run_fastkafka_server(4, app)

print("ok")

app='main:kafka_app'
[003]: [INFO] main: check
[003]: [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpepdntixw/asyncapi/spec/asyncapi.yml' does not exist.
[003]: [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpepdntixw/asyncapi/spec/asyncapi.yml'
[003]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[003]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[003]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[002]: [INFO] main: check
[003]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[003]: [INFO] fastkafka._components.aiokafka_consumer_loop: 

[000]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'group_id': 'tvrtko-fastkafka-kafka-1:9092_group', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[000]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[000]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'group_id': 'tvrtko-fastkafka-kafka-1:9092_group', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[001]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[000]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[000]: [INFO] aiokafka.consumer.subscription_state:

[003]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group tvrtko-fastkafka-kafka-1:9092_group with generation 67
[INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions {TopicPartition(topic='training_data', partition=0)} for group tvrtko-fastkafka-kafka-1:9092_group
[002]: [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[001]: [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[003]: [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[000]: [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[000]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fa